In [ ]:
import open3d as o3d
from o3d_utils import visualize, generate_lineset_from_mesh, create_ground_plane, rays_tensor, ray_geometries

%load_ext autoreload
%autoreload 2

In [ ]:
# Load the .obj file as a triangle mesh
mesh = o3d.io.read_triangle_mesh("data/NYC.obj")

mesh_edges = generate_lineset_from_mesh(mesh, exclude_diagonals=True)
ground_plane = create_ground_plane(size=500, grid_step=10)

In [13]:
# point at (0, 0, 0)
point = o3d.geometry.PointCloud()
point.points = o3d.utility.Vector3dVector([[0, 0, 0]])
point.colors = o3d.utility.Vector3dVector([[1, 0, 0]])

visualize([mesh, mesh_edges, ground_plane, point])

In [ ]:
import gnss_lib_py as glp
from datetime import datetime, timezone
import numpy as np
from sv_utils import azel2los_vec, enu2o3d

# Send time at which SV states are needed in GPS millis
timestamp_start = datetime(year=2023, month=3, day=14, hour=12, tzinfo=timezone.utc)
timestamp_end = datetime(year=2023, month=3, day=14, hour=13, tzinfo=timezone.utc)
gps_millis = glp.datetime_to_gps_millis(np.array([timestamp_start,timestamp_end]))

sp3_path = glp.load_ephemeris(file_type="sp3", gps_millis=gps_millis, verbose=True)
sp3 = glp.Sp3(sp3_path)

In [ ]:
rx_LLA = np.reshape([40.74685, -73.9895, 0], [3, 1])
rx_ecef = np.reshape(glp.geodetic_to_ecef(rx_LLA), [3, 1])

rx_state = glp.NavData()
rx_state["gps_millis"] = gps_millis[0]
rx_state["x_rx_m"] = rx_ecef[0]
rx_state["y_rx_m"] = rx_ecef[1]
rx_state["z_rx_m"] = rx_ecef[2]

# Plot skyplot computes az/els
# sp3 = sp3.where("gps_millis", sp3['gps_millis'][0], "eq")
fig = glp.plot_skyplot(sp3, rx_state)

In [11]:
timestamps = []
sv_azels_per_epoch = []

for timestamp, delta_t, data in glp.loop_time(sp3, "gps_millis"):
    all_sv_els = data['el_sv_deg']
    all_sv_azs = data['az_sv_deg']

    above_horizon_svs = all_sv_els > 0 
    above_horizon_sv_els = all_sv_els[above_horizon_svs]
    above_horizon_sv_azs = all_sv_azs[above_horizon_svs]

    azels_rad = np.radians(np.stack([above_horizon_sv_azs, above_horizon_sv_els], axis=1))
    timestamps.append(timestamp)
    sv_azels_per_epoch.append(azels_rad)

In [ ]:
mask = data["gnss_id"] == "gps"
data_df = data.pandas_df()
data_df[mask]

In [ ]:
new_data = glp.NavData()
new_data.from_pandas_df(data_df[mask])
dop_df = glp.get_dop(new_data)
dop_df["HDOP"]

In [14]:
first_gps_millis = sp3['gps_millis'][0]
all_sv_els = sp3['el_sv_deg'][sp3['gps_millis'] == first_gps_millis]
all_sv_azs = sp3['az_sv_deg'][sp3['gps_millis'] == first_gps_millis]

visible_svs = all_sv_els > 0 
visible_sv_els = all_sv_els[visible_svs]
visible_sv_azs = all_sv_azs[visible_svs]

azels_rad = np.radians(np.stack([visible_sv_azs, visible_sv_els], axis=1))

In [15]:
nominal_directions = enu2o3d(azel2los_vec(azels_rad))

SOURCE_DIST = 1000

ray_origins = SOURCE_DIST * nominal_directions
ray_directions = -ray_origins
ray_tensor = rays_tensor(ray_origins, ray_directions)
ray_geom = ray_geometries(ray_origins, ray_directions)

In [16]:
visualize([mesh, mesh_edges, ground_plane, point] + ray_geom)

In [ ]:
scene = o3d.t.geometry.RaycastingScene()
scene.add_triangles(o3d.t.geometry.TriangleMesh.from_legacy(mesh))
result = scene.cast_rays(ray_tensor)

In [ ]:
result['t_hit'].cpu().numpy() == np.inf

In [ ]:
import open3d as o3d
import numpy as np
import time

# Create a static mesh (example: a sphere)
mesh = o3d.geometry.TriangleMesh.create_sphere(radius=1.0)
mesh.paint_uniform_color([0.1, 0.1, 0.7])

# Define the initial line set
# Example: creating some simple lines
points = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1]])
lines = np.array([[0, 1], [0, 2], [0, 3]])
colors = np.array([[1, 0, 0] for _ in range(len(lines))])  # Red color for lines

line_set = o3d.geometry.LineSet()
line_set.points = o3d.utility.Vector3dVector(points)
line_set.lines = o3d.utility.Vector2iVector(lines)
line_set.colors = o3d.utility.Vector3dVector(colors)

# Create visualizer object
vis = o3d.visualization.Visualizer()
vis.create_window()

# Add static mesh to the scene
vis.add_geometry(mesh)
# Add lines to the scene
vis.add_geometry(line_set)

# Animation loop
for i in range(100):
    # Update line positions (e.g., move lines over time)
    points[:, 0] += 0.01  # Move the lines along the x-axis
    line_set.points = o3d.utility.Vector3dVector(points)
    
    vis.update_geometry(line_set)
    vis.poll_events()
    vis.update_renderer()

    time.sleep(0.05)  # Add a short delay to slow down the animation

# Close the visualizer after the animation is done
vis.destroy_window()
